In [1]:
using Pkg
pkg"activate ."

 Activating environment at `~/Documents/otimizacao-em-julia/modelagem/Project.toml`


In [2]:
using Plots
plot(rand(3))
nothing

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [76]:
using Random
Random.seed!(0)

n = 1000
v = rand(10:100, n)
p = ceil.(Int, rand(0.8:0.01:1.2, n) .* v)
C = round(Int, sum(p) * 0.5)

27994

In [77]:
[1:n  v   p  v ./ p]

1000×4 Array{Float64,2}:
    1.0   78.0   66.0  1.18182
    2.0   21.0   20.0  1.05
    3.0   53.0   49.0  1.08163
    4.0   19.0   17.0  1.11765
    5.0   94.0   79.0  1.18987
    6.0   63.0   57.0  1.10526
    7.0   86.0   82.0  1.04878
    8.0   94.0   87.0  1.08046
    9.0   45.0   53.0  0.849057
   10.0   42.0   50.0  0.84
   11.0   45.0   48.0  0.9375
   12.0   83.0   96.0  0.864583
   13.0   47.0   40.0  1.175
    ⋮                  
  989.0   37.0   31.0  1.19355
  990.0   43.0   47.0  0.914894
  991.0   33.0   35.0  0.942857
  992.0   22.0   19.0  1.15789
  993.0  100.0  106.0  0.943396
  994.0   30.0   27.0  1.11111
  995.0   81.0   76.0  1.06579
  996.0   57.0   46.0  1.23913
  997.0   17.0   15.0  1.13333
  998.0   44.0   50.0  0.88
  999.0   96.0  106.0  0.90566
 1000.0   87.0   72.0  1.20833

In [30]:
using Combinatorics

function brute(v, p, C)
    n = length(v)
    bestJ = []
    bestv = 0
    for k = 1:n
        for J in combinations(1:n, k)
            if sum(p[J]) ≤ C
                sumv = sum(v[J])
                if sumv > bestv
                    bestv = sumv
                    bestJ = copy(J)
                end
            end
        end
    end
    return bestJ, bestv, sum(p[bestJ])
end

brute (generic function with 1 method)

In [31]:
brute(v, p, C)

([1, 3, 5, 6, 7, 8, 11, 12, 13, 17, 18, 20], 328, 169)

In [46]:
function greedy(v, p, C)
    n = length(v)
    r = v ./ p
    J = sortperm(r, rev=true)
    S = Int[]
    sump = 0
    sumv = 0
    for i = 1:n
        if sump + p[J[i]] ≤ C # Se cabe na mochila
            sump += p[J[i]]
            sumv += v[J[i]]
            push!(S, J[i])
        end
        if sump == C
            break
        end
    end
    return sort(S), sumv, sump
end

greedy (generic function with 1 method)

In [47]:
greedy(v, p, C)

([1, 3, 5, 7, 8, 10, 11, 12, 13, 14, 17, 18, 20], 327, 171)

In [48]:
brute(v, p, C)

([1, 3, 5, 6, 7, 8, 11, 12, 13, 17, 18, 20], 328, 169)

In [69]:
using JuMP, Cbc

function modelo(v, p, C)
    n = length(v)
    itens = 1:n
    
    model = Model(Cbc.Optimizer)
    @variable(model, x[i ∈ itens], Bin)
    @objective(model, Max, sum(x[i] * v[i] for i ∈ itens))
    @constraint(model, sum(x[i] * p[i] for i ∈ itens) ≤ C)
    
    optimize!(model)
    
    x = [value(x[i]) for i ∈ itens]
    x = round.(Int, x)
    return findall(x .== 1), objective_value(model), sum(x .* p)
end

modelo (generic function with 1 method)

In [78]:
modelo(v, p, C)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 30487.6 - 0.00 seconds
Cgl0004I processed model has 1 rows, 840 columns (840 integer (720 of which binary)) and 840 elements
Cbc0012I Integer solution of -30413 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0012I Integer solution of -30442 found by DiveCoefficient after 39 iterations and 0 nodes (0.05 seconds)
Cbc0031I 3 added rows had average density of 840
Cbc0013I At root node, 3 cuts changed objective from -30487.619 to -30487.547 in 10 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.004 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 23 row cuts average 830.1 elements, 0 column cuts (0 active)  in 0.005 seconds - new frequency is -100
Cbc0014I Cut generator 2 (Knapsack) - 0 row cuts average 0.0 elements, 0 c

([1, 2, 3, 4, 5, 6, 7, 8, 13, 14  …  985, 987, 988, 989, 992, 994, 995, 996, 997, 1000], 30487.0, 27994)

In [79]:
greedy(v, p, C)

([1, 2, 3, 4, 5, 6, 7, 8, 13, 14  …  985, 987, 988, 989, 992, 994, 995, 996, 997, 1000], 30487, 27994)